In [32]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import matplotlib.font_manager as fm
import matplotlib.pyplot as plt
import matplotlib
import math
import random
import matplotlib.pyplot as plt

In [3]:
# 读取预处理过的数据
original = pd.read_excel('./第二问预处理.xlsx')

In [4]:
# 选择 2023 年 6 月 24-30 日的数据
original['销售日期'] = pd.to_datetime(original['销售日期'])
recent_data = original[(original['销售日期'] >= '2023-06-24') & (original['销售日期'] <= '2023-06-30')]

In [5]:
recent_data

,销售日期,扫码销售时间,单品编码,辅助列,销量(千克),销售单价(元/千克),销售类型,是否打折销售,分类编码,分类名称,批发价格(元/千克),损耗率(%),成本加成定价(元/千克)
875001,2023-06-24,09:10:47.276,102900011032022,45101-102900011032022,1.000,2.8,销售,否,1011010504,辣椒类,1.51,9.43,2.957219
875002,2023-06-24,09:10:47.531,102900005115786,45101-102900005115786,0.379,4.0,销售,否,1011010101,花叶类,2.41,13.62,4.379998
875003,2023-06-24,09:13:51.795,102900005115762,45101-102900005115762,0.505,4.0,销售,否,1011010101,花叶类,2.31,18.52,4.525052
875004,2023-06-24,09:14:43.77,102900005115762,45101-102900005115762,0.505,4.0,销售,否,1011010101,花叶类,2.31,18.52,4.525052
875005,2023-06-24,09:14:44.044,102900011030059,45101-102900011030059,1.000,4.5,销售,否,1011010101,花叶类,3.32,9.43,4.845673
...,...,...,...,...,...,...,...,...,...,...,...,...,...
878498,2023-06-30,21:35:13.264,102900005115250,45107-102900005115250,0.284,24.0,销售,否,1011010801,食用菌,15.60,10.80,25.888789
878499,2023-06-30,21:35:14.358,102900011022764,45107-102900011022764,0.669,12.0,销售,否,1011010501,茄类,7.00,6.90,12.518797
878500,2023-06-30,21:35:20.264,102900005115250,45107-102900005115250,0.125,24.0,销售,否,1011010801,食用菌,15.60,10.80,25.888789
878501,2023-06-30,21:35:21.509,102900011016701,45107-102900011016701,0.252,5.2,销售,否,1011010504,辣椒类,3.63,5.70,5.419417


In [6]:
# 去除重复值并获取去重后的单品编码列表
unique_codes = recent_data['单品编码'].drop_duplicates().unique()

# 计算去重后的单品编码个数
num_unique_codes = len(unique_codes)

print(unique_codes)
print(num_unique_codes)

# 创建一个DataFrame对象
df = pd.DataFrame(unique_codes, columns=['单品编码'])

# 导出为Excel文件
df.to_excel('./results/49个符合的单品.xlsx', index=False)



[102900011032022 102900005115786 102900005115762 102900011030059
 102900005115823 102900011006948 102900005115250 102900051000463
 102900011016701 102900011030097 102900011035078 106971533450003
 102900005116714 102900005116899 102900011031100 102900005119975
 102900005116257 102900011034026 102900011008164 102900005115908
 102900005118831 102900011031926 102900011022764 106949711300259
 102900011023464 102900005115946 102900011034330 102900005118824
 102900051000944 102900005116509 102900011032251 102900011000328
 102900005118817 102900011035740 102900011001691 102900011032848
 102900011032343 102900011030110 102900011007969 102900011034439
 102900011032237 102900005115984 102900011032732 102900005115779
 102900011013274 102900011030929 102900011018132 102900011033982
 102900011036686]
49


In [22]:
# 读取预处理过的数据
original_1 = pd.read_excel('./第三问top33.xlsx')

In [23]:
# 根据单品求损耗率
def compute_rate(danpin):
    
    sunhao = original_1[original['单品编码'] == danpin]['损耗率']
    return sunhao.iloc[0]*0.01

In [24]:
# 给单品和进货量-求出陈列量
def compute_chenlie(danpin,jinhuo):
    chenlie=jinhuo*(1-compute_rate(danpin))

    return chenlie

In [25]:
# 给单品-求进价
def compute_jinjia(danpin):
    sunhao = original_1[original_1['单品编码'] == danpin]['平均进价']
    return sunhao.iloc[0]

In [30]:
# 给单品-求成本加成定价
def compute_dingjia(danpin):
    sunhao = original_1[original_1['单品编码'] == danpin]['修正后成本加成定价预测']
    return sunhao.iloc[0]

In [31]:
compute_dingjia(102900011030059)

5.223263

In [18]:
# 给出单品编码、进货量-求出该商品的利润
def compute_lirun(danpin,jinhuo):
    lirun=compute_chenlie(danpin,jinhuo)*compute_dingjia(danpin)-jinhuo*compute_jinjia(danpin)

    return lirun

In [33]:
x = 0
a = 0.97  # 退火速度
T = 100  # 初始温度
min_quanju = T  # 全局最优解初始化
min_x = 0  # 全局最优解对应X坐标
n = 0  # 退火计次
listn = []
list_min = []

In [ ]:
while T > 1:  # 终止条件

    min_jubu = T
    t = T
    min_x1 = random.randint(0, 500)/100

    for i in range(2):
        x1 = min_x1 + random.randint(-100, 100)/100
        y1 = mbhs(x1)
        dt = y1 - min_jubu
        if dt < 0 or dt == 0:
            t = y1
            min_x1 = x1
        else:
            zhishu = -dt/t
            met = math.exp(zhishu)
            pd = random.randint(0, 1000000)/1000000
            if pd <= met:
                t = y1
                min_x1 = x1
        min_jubu = t

    if min_jubu <= min_quanju:
        min_quanju = min_jubu
        min_x = min_x1

    print('局部最优解：', min_jubu)
    T = T * a

    n += 1
    listn.append(T)
    list_min.append(min_jubu)

print('全局最优解：', min_quanju)
print('最优解时X取值:', min_x)

# 制图
X = list_x = []
Y = list_y = []
c = 0
for i in range(500):
    list_x.append(i/100)
    list_y.append(mbhs(i/100))

#图中使用汉字
plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False

#最优解绘图
plt.subplot(121)
plt.plot(X, Y)
plt.scatter(min_x, min_quanju, c='r', label='全局最优解')
plt.legend()

#退火过程
plt.subplot(122)
plt.plot(listn, list_min)
plt.gca().invert_xaxis()

plt.show()